<a href="https://colab.research.google.com/github/heirynhr/legalduel-1a/blob/main/LegalDuel1A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install dateparser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 7.3 MB/s eta 0:00:00


In [3]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [4]:

!pip install nltk


In [24]:
# importing libraries here

from transformers import BartTokenizer, BartForConditionalGeneration
import torch
from PyPDF2 import PdfReader
from google.colab import files
import spacy
import nltk
from nltk.corpus import stopwords
from datetime import datetime
import dateparser
from transformers import pipeline
nltk.download('stopwords')

nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# this is how we are able to upload the different chronologies into the document
# run this cell to add new docs

uploaded = files.upload()

Saving Chronology1official.pdf to Chronology1official.pdf


In [7]:
# looking at our input file here, only used for looking at document, not as input
# for model


filepath = "Chronology1official.pdf"
reader = PdfReader(filepath)
text = ""
numPages = len(reader.pages)

for pageNum in range(numPages):
  page = reader.pages[pageNum]
  text = text + page.extract_text()
  print("Page:" + str(pageNum + 1))

print(text)

Page:1
Page:2
Page:3
Page:4
Page:5
Page:6
**Deposition of Erna Wiggins****Date:** August 1, 2024  **Time:** 9:00 AM  **Location:** Hawkins LLP, Conference Room B**Attorneys Present:**- **For Plaintiff:** Ryan Backstroem, Esq.- **For Defendant:** Jennifer Ryan, Esq.**Court Reporter:** Sarah Johnson---**Ryan Backstroem, Esq.:** Good morning, Ms. Wiggins. Could you please state your full name for the record?**Erna Wiggins:** My name is Erna Marie Wiggins.**Ryan Backstroem, Esq.:** Thank you. Ms. Wiggins, where do you currently reside?**Erna Wiggins:** I live at 123 Maple Street, Philadelphia, Pennsylvania.**Ryan Backstroem, Esq.:** Let's start with your employment history. Where were you employed on January 15, 2020?**Erna Wiggins:** On January 15, 2020, I was employed at Tech Solutions Inc. as a Senior Project Manager.**Ryan Backstroem, Esq.:** And how long had you been with Tech Solutions Inc. by that date?**Erna Wiggins:** I started working there on March 1, 2015.**Ryan Backstroem, Esq

In [7]:
# clean text function

def clean_text(text):
  nlpText = nlp(text)

  cleaned_text = []

  for word in nlpText:
    if not word.is_stop and not word.is_punct:
      cleaned_text.append(word.lemma_.lower())

  return " ".join(cleaned_text)

print(clean_text(text))

deposition erna wiggins****date august 1 2024   time 9:00   location hawkins llp conference room b**attorney present:**- plaintiff ryan backstroem esq.- defendant jennifer ryan esq.**court reporter sarah johnson---**ryan backstroem esq good morning ms. wiggins state record?**erna wiggins erna marie wiggins.**ryan backstroem esq thank ms. wiggins currently reside?**erna wiggins live 123 maple street philadelphia pennsylvania.**ryan backstroem esq let start employment history employ january 15 2020?**erna wiggin january 15 2020 employ tech solutions inc. senior project manager.**ryan backstroem esq long tech solutions inc. date?**erna wiggins start work march 1 2015.**ryan backstroem esq describe responsibility tech solutions inc.?**erna wiggin responsible oversee project timeline manage team 10 ensure project complete schedule budget.**ryan backstroem esq involved speciﬁc project 2020?**erna wiggins yes lead development new client management software start february 10 2020.**ryan backst

In [8]:
# extracting text for the model

def extract_text_from_pdf(filepath):
  reader = PdfReader(filepath)
  text = ""

  for i in range(len(reader.pages)):
    page = reader.pages[i]
    pageText = page.extract_text()

    pageText = pageText.replace("\n", " ").strip()
    text += pageText + "\n\n"

    final_text = clean_text(text)
  return text

In [9]:
# summarzing text function
def summarize_transcript(text, max_length=1024):
  inputs = tokenizer(text, max_length=max_length, return_tensors="pt", truncation=True)
  summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return summary

In [10]:
def extract_events(text):
    doc = nlp(text)
    events = []
    current_date = None

    for sent in doc.sents:

        for ent in sent.ents:
            if ent.label_ == "DATE":
                current_date = ent.text
                break


        if current_date:
            event = f"{current_date}: {sent.text.strip()}"
            events.append(event)
            current_date = None

    return events

In [11]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

def create_chronology(filepath):
  transcript = extract_text_from_pdf(filepath)
  summary = summarize_transcript(transcript)
  return summary

chronology = create_chronology(filepath)
print(chronology)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Erna Wiggins was employed at Tech Solutions Inc. as a Senior Project Manager. She was responsible for overseeing project timelines, managing a team of 10, and ensuring that all projects were completed on schedule and within budget. The project was eventually completed on March 15, 2021.


---- Here is where we will be experimenting with different date recognition tools

In [8]:
nlp = spacy.load("en_core_web_sm")

doc = nlp(text)

# Extracting dates from the raw text
dates = [ent for ent in doc.ents if ent.label_ in ("DATE")]
print(dates)

[August 1, 2024  , January 15, January 15, 2020, March 1, February 10, December 1, March 15, April 5, 2020, June 20, April 7, 2020, April 15, July 10, September 1, 2020, October 15, 2020, weekly, every Monday, September 2, 2020, March 15, period?**Erna, November 5, 2020, December 10, 2020, January 20, February 5, 2021, February 10, 2021, March 10, February 25, 2021, March 1, March 15, December 10, December 12, 2020, December 20]


In [13]:
# Stripping the text and storing in a list
date_strs = [date.text.strip() for date in dates]
date_strs

['August 1, 2024',
 'January 15',
 'January 15, 2020',
 'March 1',
 'February 10',
 'December 1',
 'March 15',
 'April 5, 2020',
 'June 20',
 'April 7, 2020',
 'April 15',
 'July 10',
 'September 1, 2020',
 'October 15, 2020',
 'weekly',
 'every Monday',
 'September 2, 2020',
 'March 15',
 'period?**Erna',
 'November 5, 2020',
 'December 10, 2020',
 'January 20',
 'February 5, 2021',
 'February 10, 2021',
 'March 10',
 'February 25, 2021',
 'March 1',
 'March 15',
 'December 10',
 'December 12, 2020',
 'December 20']

In [15]:
# Where we are storing the date objects
date_objs = []

# Iterating through each date
for date in date_strs:
  try:
    # Try to format the date with the year included
    # For some reason we are having issues with this one
    date_obj = datetime.strptime(date, "B% %d, %Y")
  except ValueError:
    try:
      # If no year, try to format the date with just month and day
      date_obj = datetime.strptime(date, "%B %d")
    except ValueError:
      # If it's not a date the error will be handled here
      print(f"Ignoring invalid date string: {date}")
      continue
  # Only adding valid date objects to the list
  date_objs.append((date_obj, date))

# Sorting, right now we don't have year format working so it is by month
sorted_dates = sorted(date_objs)
sorted_dates

Ignoring invalid date string: August 1, 2024
Ignoring invalid date string: January 15, 2020
Ignoring invalid date string: April 5, 2020
Ignoring invalid date string: April 7, 2020
Ignoring invalid date string: September 1, 2020
Ignoring invalid date string: October 15, 2020
Ignoring invalid date string: weekly
Ignoring invalid date string: every Monday
Ignoring invalid date string: September 2, 2020
Ignoring invalid date string: period?**Erna
Ignoring invalid date string: November 5, 2020
Ignoring invalid date string: December 10, 2020
Ignoring invalid date string: February 5, 2021
Ignoring invalid date string: February 10, 2021
Ignoring invalid date string: February 25, 2021
Ignoring invalid date string: December 12, 2020


[(datetime.datetime(1900, 1, 15, 0, 0), 'January 15'),
 (datetime.datetime(1900, 1, 20, 0, 0), 'January 20'),
 (datetime.datetime(1900, 2, 10, 0, 0), 'February 10'),
 (datetime.datetime(1900, 3, 1, 0, 0), 'March 1'),
 (datetime.datetime(1900, 3, 1, 0, 0), 'March 1'),
 (datetime.datetime(1900, 3, 10, 0, 0), 'March 10'),
 (datetime.datetime(1900, 3, 15, 0, 0), 'March 15'),
 (datetime.datetime(1900, 3, 15, 0, 0), 'March 15'),
 (datetime.datetime(1900, 3, 15, 0, 0), 'March 15'),
 (datetime.datetime(1900, 4, 15, 0, 0), 'April 15'),
 (datetime.datetime(1900, 6, 20, 0, 0), 'June 20'),
 (datetime.datetime(1900, 7, 10, 0, 0), 'July 10'),
 (datetime.datetime(1900, 12, 1, 0, 0), 'December 1'),
 (datetime.datetime(1900, 12, 10, 0, 0), 'December 10'),
 (datetime.datetime(1900, 12, 20, 0, 0), 'December 20')]

In [47]:
# Parameter for how many words we're looking at when summarizing the text
# around a date
context_window = 25

# Where we are storing the date tokens
date_tokens = []

for date_obj, original_str in sorted_dates:
  for ent in doc.ents:
    if ent.label_ == "DATE" and ent.text == original_str:
      # Setting endpoints
      date_start = ent.start
      date_end = ent.end

      # Setting the windows for the token text
      start_context = max(0, date_start - context_window)
      end_context = min(len(doc), date_end + context_window)

      # Slicing the part of text pertaining to the date and then adding to the list
      context_tokens = doc[start_context:end_context]
      date_tokens.append((context_tokens, date_obj, original_str))
      break

date_tokens

[(Maple Street, Philadelphia, Pennsylvania.**Ryan Backstroem, Esq.:** Let's start with your employment history. Where were you employed on January 15, 2020?**Erna Wiggins:** On January 15, 2020, I was employed at Tech Solutions Inc. as a Senior Project Manager.**Ryan Backstroem,,
  datetime.datetime(1900, 1, 15, 0, 0),
  'January 15'),
 (.:** How did you handle the client feedback?**Erna Wiggins:** We incorporated the changes into our development plan and completed them by January 20, 2021.**Ryan Backstroem, Esq.:** Did you have any other signiﬁcant interactions with clients?**Erna Wiggins:** Yes, we had a,
  datetime.datetime(1900, 1, 20, 0, 0),
  'January 20'),
 (speciﬁc projects in 2020?**Erna Wiggins:** Yes, I was leading the development of the new client management software, which we started on February 10, 2020.**Ryan Backstroem, Esq.:** When was this project scheduled to be completed?**Erna Wiggins:** The initial deadline was set for,
  datetime.datetime(1900, 2, 10, 0, 0),
  'F

In [50]:
def summarize_date_tokens(tokens):
  summaries = []
  summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
  for context, date_obj, date_str in tokens:

    context_text = context.text

    summary = summarizer(context_text, max_length=40, min_length=5, do_sample=False)

    # Formatting to exclude extra built in text
    summary_strip = summary[0]
    summary_text = summary_strip['summary_text']

    event = f"{date_str}: {summary_text}"

    summaries.append(event)

  return summaries

summaries = summarize_date_tokens(date_tokens)
summaries

["January 15:  On January 15, 2020, Erna Wiggins was employed at Tech Solutions Inc. as a Senior Project Manager . Ryan Backstroem, Esq.:** Let's start with your employment",
 'January 20:  Erna Wiggins: We incorporated the changes into our development plan and completed them by January 20, 2021 . Ryan Backstroem, Esq.:** Did you have any other signi',
 'February 10:  Erna Wiggins was leading the development of the new client management software . The initial deadline was set for February 10, 2020 .',
 'March 1:  Erna Wiggins:** I started working there on March 1, 2015 . I was responsible for overseeing Tech Solutions Inc.',
 'March 1:  Erna Wiggins:** I started working there on March 1, 2015 . I was responsible for overseeing Tech Solutions Inc.',
 'March 10:  Ryan Backstroem, Esq.: When did the ﬁnal testing phase begin? Erna Wiggins:** It began on February 10, 2021, and lasted until March',
 'March 15:  The project was eventually completed on March 15, 2021 . Erna Wiggins: "We faced 

In [33]:
print(type(summaries[0]))

<class 'str'>
